In [1]:
import pandas as pd

In [2]:
dt = pd.read_csv("ratings.csv")

In [3]:
dt.head()

,User_ID,Food_ID,Rating
0,1.0,88.0,4.0
1,1.0,46.0,3.0
2,1.0,24.0,5.0
3,1.0,25.0,4.0
4,2.0,49.0,1.0


In [4]:
dt.tail()

,User_ID,Food_ID,Rating
507,99.0,22.0,1.0
508,100.0,24.0,10.0
509,100.0,233.0,10.0
510,100.0,29.0,7.0
511,NaN,NaN,NaN


In [5]:
dt.drop(dt.tail(1).index,inplace=True)

In [6]:
dt.tail()

,User_ID,Food_ID,Rating
506,99.0,65.0,7.0
507,99.0,22.0,1.0
508,100.0,24.0,10.0
509,100.0,233.0,10.0
510,100.0,29.0,7.0


In [7]:
dt.loc[len(dt.index)] = [101, 24, 9] 

# dt2 = {'Name': 101, 'Maths': 24, 'Science': 9}
# dt = dt.append(dt2, ignore_index = True)
# https://www.stackvidhya.com/add-row-to-dataframe/
dt.tail()

,User_ID,Food_ID,Rating
507,99.0,22.0,1.0
508,100.0,24.0,10.0
509,100.0,233.0,10.0
510,100.0,29.0,7.0
511,101.0,24.0,9.0


In [8]:
dt2 = dt.drop('User_ID',1)

In [9]:
dt3 = dt2.groupby('Food_ID').mean()
dt3.head()

,Rating
Food_ID,
1.0,7.5
2.0,6.0
3.0,4.0
4.0,6.0
5.0,6.5


In [10]:
flt = dt['Food_ID'] == 1
dt.loc[flt]['Rating'].mean()

7.5

In [11]:
food = pd.read_csv("food.csv")
ratings = pd.read_csv("ratings.csv")
dataset = pd.read_csv("users_combined_rating_pivot.csv")
preds = pd.read_csv("predicted.csv")

In [12]:
preds.head()

,Unnamed: 0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,300.0,301.0,302.0,303.0,304.0,305.0,306.0,307.0,308.0,309.0
0,0,-0.048949,0.002101,0.011330,-0.077076,0.020332,-0.028672,-0.098129,-0.171783,-0.124071,...,-0.014799,0.060264,0.024905,0.073726,0.022573,-0.011967,0.087599,0.014540,0.252929,0.022718
1,1,-0.186800,-0.007912,-0.042540,-0.898963,-0.084581,-0.028339,-0.150486,-0.101445,-0.115920,...,0.062862,0.079658,0.050724,0.103593,0.065283,-0.166912,-0.039653,0.047565,0.030345,0.012797
2,2,-0.032185,0.019317,-0.108391,-0.063803,-0.080629,-0.027423,-0.091557,-0.053898,-0.083616,...,-0.020945,0.053987,0.057926,0.040231,0.067101,-0.134565,0.029937,0.059610,0.050380,-0.049891
3,3,-0.350374,-0.172312,-0.078371,-0.109807,-0.039780,-0.024423,-0.225578,-0.011290,-0.251320,...,0.175080,0.113178,0.057976,0.132643,0.135032,-0.304662,0.056973,0.082878,0.155382,-0.023058
4,4,-0.024147,0.001343,-0.230989,0.056057,-0.082824,-0.038933,-0.069143,6.120407,-0.082034,...,0.095883,0.056805,0.067464,0.031411,0.040180,-0.135914,0.044329,0.039137,0.075536,-0.045251


In [13]:
preds.rename(columns = {"Unnamed: 0":"Food_ID"}, inplace=True)
preds.reset_index(drop=True,inplace=True)
preds.set_index('Food_ID', inplace=True)
preds.head()

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,300.0,301.0,302.0,303.0,304.0,305.0,306.0,307.0,308.0,309.0
Food_ID,,,,,,,,,,,,,,,,,,,,,
0,-0.048949,0.002101,0.011330,-0.077076,0.020332,-0.028672,-0.098129,-0.171783,-0.124071,-0.043170,...,-0.014799,0.060264,0.024905,0.073726,0.022573,-0.011967,0.087599,0.014540,0.252929,0.022718
1,-0.186800,-0.007912,-0.042540,-0.898963,-0.084581,-0.028339,-0.150486,-0.101445,-0.115920,-0.108364,...,0.062862,0.079658,0.050724,0.103593,0.065283,-0.166912,-0.039653,0.047565,0.030345,0.012797
2,-0.032185,0.019317,-0.108391,-0.063803,-0.080629,-0.027423,-0.091557,-0.053898,-0.083616,-0.091422,...,-0.020945,0.053987,0.057926,0.040231,0.067101,-0.134565,0.029937,0.059610,0.050380,-0.049891
3,-0.350374,-0.172312,-0.078371,-0.109807,-0.039780,-0.024423,-0.225578,-0.011290,-0.251320,-0.117247,...,0.175080,0.113178,0.057976,0.132643,0.135032,-0.304662,0.056973,0.082878,0.155382,-0.023058
4,-0.024147,0.001343,-0.230989,0.056057,-0.082824,-0.038933,-0.069143,6.120407,-0.082034,-0.247019,...,0.095883,0.056805,0.067464,0.031411,0.040180,-0.135914,0.044329,0.039137,0.075536,-0.045251


In [14]:
preds.columns = preds.columns.map(float)

In [15]:
list(preds.columns)

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0,
 139

In [16]:
preds.index

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
            68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
            85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99],
           dtype='int64', name='Food_ID')

In [17]:
preds.head(20).index.tolist()

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [18]:
def svd_food_recommendation(predictions, userID, food, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the food information.
    user_data = original_ratings[original_ratings.User_ID == (userID)]
    user_full = (user_data.merge(food, how = 'left', left_on = 'Food_ID', right_on = 'Food_ID').
                     sort_values(['Rating'], ascending=False)
                 )
    
    # print('User {0} has already rated {1} food.'.format(userID, user_full.shape[0]))
    # print('Recommending highest {0} predicted ratings food not already rated.'.format(num_recommendations))
#     sorted_user_predictions.set_index('Food_ID', inplace=True)
    sorted_user_predictions_df = pd.DataFrame(sorted_user_predictions)
    sorted_user_predictions_df.index.name = "Food_ID"
    display(user_full)
    display(sorted_user_predictions_df.reset_index())
    
#     Recommend the highest predicted rating foods that the user hasn't tasted yet.
    recommendations = (food[~food['Food_ID'].isin(user_full['Food_ID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'Food_ID',
               right_on = 'Food_ID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return recommendations

# svd_food_recommendation(preds, 4, food, ratings, 10)
#data = knn_food_recommendation("chicken biryani")
#print(data)
predictions = svd_food_recommendation(preds, 4, food, ratings, 10)
predictions

,User_ID,Food_ID,Rating,Name,C_Type,Veg_Non,Describe
3,4.0,212.0,10.0,al hachi chicken,Indian,non-veg,"shallow fried chicken, bottle gourd (boiled an..."
4,4.0,128.0,6.0,thai lamb balls,Thai,non-veg,"lamb (minced), couscous, scallion, garlic, egg..."
0,4.0,14.0,5.0,"peach, raspberry and nuts smoothie",Healthy Food,veg,"fresh raspberries, ripe banana, almond, fresh ..."
1,4.0,141.0,5.0,"risotto lobster with parmesan egg pancake, con...",Italian,non-veg,"lobster shell, carrot, leeks, garlic cloves, t..."
2,4.0,170.0,1.0,fried rice with soya chunks,Chinese,veg,"basmati rice, carrot, capsicum, beans, green p..."
5,4.0,21.0,1.0,apple rabdi,Dessert,veg,"apples, milk, sugar, green cardamoms, almonds ..."


,Food_ID,3
0,212.0,9.509673
1,128.0,5.676348
2,14.0,4.737564
3,141.0,4.737564
4,170.0,1.051432
...,...,...
304,47.0,-0.422146
305,15.0,-0.454916
306,172.0,-0.471280
307,119.0,-0.471280


,Food_ID,Name,C_Type,Veg_Non,Describe
150,155,chilli fish,Chinese,non-veg,"For fish pieces (Boneless), Flour, Cornflour, ..."
58,61,crunchy vegetable dal sattu croquettes,Italian,veg,"red lentils or masoor dal (half-boiled), potat..."
93,96,methi chicken masala,Indian,non-veg,"boneless chicken, lemon juice, red chilli powd..."
180,186,strawberry & pistachio breton tart,Dessert,veg,"plain flour, baking powder, sea salt, unsalted..."
124,127,cajun spiced turkey wrapped with bacon,Mexican,non-veg,"turkey breast, cajun spice, spinach leaves (co..."
42,45,ghee roast chicken dosa quesadilla,Mexican,non-veg,"dry red chillies, coriander seeds, cumin seeds..."
262,269,spiced orange valencia cake,Dessert,veg,"egg whites, egg white powder, sugar, almond po..."
60,63,flax seed and beetroot modak,Japanese,veg,"rice flour, salt, oil (for greasing), beetroot..."
146,151,fish andlouse,French,non-veg,"white wine and water mix to cover, onion, salt..."
71,74,slow cooked spiced sangria,Mexican,veg,"red wine (merlot / zinfandel), apple cider, ho..."


In [26]:
import pickle
from scipy.sparse import csr_matrix
food = pd.read_csv("food.csv")
ratings = pd.read_csv("ratings.csv")
dataset = pd.read_csv("users_combined_rating_pivot.csv")

In [67]:
def knn_food_recommendation(Food_Name):

    dataset.set_index('Food_ID', inplace=True)
    knn_model = pickle.load(open('knn_model.pkl','rb'))
    csr_dataset = csr_matrix(dataset.values)
    dataset.reset_index(inplace=True)
    n = 5
    FoodList = food[food['Name'].str.contains(Food_Name)]  
    if len(FoodList):        
        Foodi= FoodList.iloc[0]['Food_ID']
        Foodi = dataset[dataset['Food_ID'] == Foodi].index[0]
        distances , indices = knn_model.kneighbors(csr_dataset[Foodi],n_neighbors=n+1)    
        Food_indices = sorted(list(zip(indices.squeeze().tolist(),distances.squeeze().tolist())),key=lambda x: x[1])
#         Food_indices = list(zip(indices.squeeze().tolist(),distances.squeeze().tolist()))[:0:-1]
        Recommendations = []
#        print(Food_indices)
#        print(distances)
        Given_food_index = Foodi
        for val in Food_indices:
#             if(val[1]==0.0):
#                 continue
#             print(val[0])
            Foodi = dataset.iloc[val[0]]['Food_ID']
            i = food[food['Food_ID'] == Foodi].index
            if(i.values==Given_food_index):
                n-=1
                continue
#             print('Food_id:',Foodi,' Name:',food.iloc[i]['Name'].values[0],'with a distance of',val[1])
            Recommendations.append({'Food_id':Foodi,'Name':food.iloc[i]['Name'].values[0],'Distance':val[1]})
        df = pd.DataFrame(Recommendations,index=range(0,n+1))
        return df
    else:
        return "No Similar Foods."

In [68]:
knn_food_recommendation("chicken biryani")

,Food_id,Name,Distance
0,182.0,avial with red rice,0.071523
1,60.0,caramelized sesame smoked almonds,0.071523
2,180.0,lotus leaf wrapped fried rice,0.071523
3,244.0,jalebi with fennel yogurt pudding,0.326246
4,184.0,vegetable bruschetta,0.590422


In [69]:
knn_food_recommendation("avial with red rice")

,Food_id,Name,Distance
0,180.0,lotus leaf wrapped fried rice,0.000000
1,60.0,caramelized sesame smoked almonds,0.000000
2,91.0,chicken biryani,0.071523
3,184.0,vegetable bruschetta,0.558871
4,244.0,jalebi with fennel yogurt pudding,0.648877
